<a href="https://colab.research.google.com/github/chw8207/pytorch_study/blob/main/softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
torch.manual_seed(1)

###  파이토치로 소프트맥스의 비용 함수 구현하기 (로우-레벨)

In [ ]:
z = torch.FloatTensor([1,2,3])

In [ ]:
# 텐서를 소프트맥스 함수 입력으로 사용하고 확인해보기
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [ ]:
# 원소들의 값의 합이 1인지 확인해보기
hypothesis.sum()

tensor(1.)

In [ ]:
# 임의의 3X5행렬의 크기를 가진 텐서 선언
z = torch.rand(3,5, requires_grad=True)

In [ ]:
# 텐서에 소프트맥스함수 적용
# 텐서 차원이 2차원이라 두 번째 차원 : dim=1
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [ ]:
# 각 샘플에 대한 임의의 레이블 만들기
y = torch.randint(5,(3,)).long()
print(y)

tensor([2, 1, 0])


In [ ]:
# 모든 원소가 0의 값을 가진 3 × 5 텐서 생성
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1,y.unsqueeze(1),1)

tensor([[0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.]])

In [ ]:
# 비용함수 구현하기
cost = (y_one_hot*-torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.6682, grad_fn=<MeanBackward0>)


###  파이토치로 소프트맥스의 비용 함수 구현하기 (하이-레벨)

In [ ]:
(y_one_hot*-F.log_softmax(z,dim=1)).sum(dim=1).mean()

tensor(1.6682, grad_fn=<MeanBackward0>)

In [ ]:
# F.nll_loss()를 사용할 때는 원-핫 벡터를 넣을 필요없이 바로 실제값을 인자로 사용
# High level
F.nll_loss(F.log_softmax(z, dim=1),y)

tensor(1.6682, grad_fn=<NllLossBackward0>)

In [ ]:
# F.log_softmax() + F.nll_loss() = F.cross_entropy()
F.cross_entropy(z,y)

tensor(1.6682, grad_fn=<NllLossBackward0>)

### 소프트맥스 회귀 구현하기
 - 로우-레벨, F.cross_entropy

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

In [3]:
# 훈련 데이터와 레이블을 텐서로 선언
X_train = [[1,2,1,1],
           [2,1,3,2],
           [3,1,3,4],
           [4,1,5,5],
           [1,7,5,5],
           [1,2,5,6],
           [1,6,6,6],
           [1,7,7,7]]
y_train = [2,2,2,1,1,1,0,0]

X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train)

In [4]:
# 소프트맥스회귀(로우-레벨)
# 크기 확인
print(X_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


In [11]:
y_one_hot = torch.zeros(8,3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1).type(torch.int64)

RuntimeError: ignored

In [13]:
# 소프트맥스회귀(하이-레벨)
# 모델 초기화
W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W,b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs+1) : 
  # cost 계산
  z = X_train.matmul(W) + b
  cost = F.cross_entropy(z, y_train.long())

  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch%100==0 : 
    print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}')

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568256
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


### 소프트맥스 회귀 nn.Module로 구현
 - output_dim : 클래스의 개수

In [14]:
# 모델 선언 및 초기화
# 4개의 특성을 가지고 3개의 클래스로 분류
model = nn.Linear(4,3)

In [16]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs+1) : 
  # H(x) 계산
  prediction = model(X_train)

  # cost 계산
  cost = F.cross_entropy(prediction, y_train.long())

  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch%20==0 : 
    print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}')

Epoch    0/1000 Cost: 1.616785
Epoch   20/1000 Cost: 0.904600
Epoch   40/1000 Cost: 0.778017
Epoch   60/1000 Cost: 0.722806
Epoch   80/1000 Cost: 0.686341
Epoch  100/1000 Cost: 0.658891
Epoch  120/1000 Cost: 0.636713
Epoch  140/1000 Cost: 0.617946
Epoch  160/1000 Cost: 0.601539
Epoch  180/1000 Cost: 0.586845
Epoch  200/1000 Cost: 0.573443
Epoch  220/1000 Cost: 0.561047
Epoch  240/1000 Cost: 0.549451
Epoch  260/1000 Cost: 0.538507
Epoch  280/1000 Cost: 0.528102
Epoch  300/1000 Cost: 0.518151
Epoch  320/1000 Cost: 0.508588
Epoch  340/1000 Cost: 0.499357
Epoch  360/1000 Cost: 0.490416
Epoch  380/1000 Cost: 0.481729
Epoch  400/1000 Cost: 0.473265
Epoch  420/1000 Cost: 0.465000
Epoch  440/1000 Cost: 0.456911
Epoch  460/1000 Cost: 0.448979
Epoch  480/1000 Cost: 0.441185
Epoch  500/1000 Cost: 0.433516
Epoch  520/1000 Cost: 0.425956
Epoch  540/1000 Cost: 0.418493
Epoch  560/1000 Cost: 0.411114
Epoch  580/1000 Cost: 0.403807
Epoch  600/1000 Cost: 0.396563
Epoch  620/1000 Cost: 0.389370
Epoch  6

### 소프트맥스 회귀 클래스로 구현

In [17]:
class SoftMaxClassifierModel(nn.Module) : 
  def __init__(self) : 
    super().__init__()
    self.linear = nn.Linear(4,3)

  def forward(self, X) : 
    return self.linear(X)

In [18]:
model = SoftMaxClassifierModel()

In [21]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs+1) : 
  # H(x) 계산
  prediction = model(X_train)

  # cost 계산
  cost = F.cross_entropy(prediction, y_train.long())

  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch%20==0 : 
    print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}')

Epoch    0/1000 Cost: 1.476262
Epoch   20/1000 Cost: 0.950686
Epoch   40/1000 Cost: 0.832654
Epoch   60/1000 Cost: 0.771019
Epoch   80/1000 Cost: 0.732903
Epoch  100/1000 Cost: 0.705964
Epoch  120/1000 Cost: 0.684907
Epoch  140/1000 Cost: 0.667274
Epoch  160/1000 Cost: 0.651812
Epoch  180/1000 Cost: 0.637822
Epoch  200/1000 Cost: 0.624880
Epoch  220/1000 Cost: 0.612713
Epoch  240/1000 Cost: 0.601137
Epoch  260/1000 Cost: 0.590021
Epoch  280/1000 Cost: 0.579271
Epoch  300/1000 Cost: 0.568815
Epoch  320/1000 Cost: 0.558599
Epoch  340/1000 Cost: 0.548581
Epoch  360/1000 Cost: 0.538727
Epoch  380/1000 Cost: 0.529011
Epoch  400/1000 Cost: 0.519410
Epoch  420/1000 Cost: 0.509906
Epoch  440/1000 Cost: 0.500485
Epoch  460/1000 Cost: 0.491133
Epoch  480/1000 Cost: 0.481839
Epoch  500/1000 Cost: 0.472595
Epoch  520/1000 Cost: 0.463391
Epoch  540/1000 Cost: 0.454222
Epoch  560/1000 Cost: 0.445080
Epoch  580/1000 Cost: 0.435962
Epoch  600/1000 Cost: 0.426860
Epoch  620/1000 Cost: 0.417772
Epoch  6